In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re

In [30]:
# set path to experimental data
data_dir = '../data/test_files/experimental/'
file_path_reads_left = data_dir + 'left.1.sam'
file_path_reads_right = data_dir + 'right.2.sam'

In [65]:
# load sam files for bound 8 8, left and right reads
# skip header
# only read in first 10 columns
# read in the left and right reads line by line
with open(file_path_reads_left, 'r') as reads_file_left:
    reads_left = reads_file_left.readlines()
    # remove the new line character at the end of each line
    reads_left = [read.strip() for read in reads_left]

with open(file_path_reads_right, 'r') as reads_file_right:
    reads_right = reads_file_right.readlines()
    # remove the new line character at the end of each line
    reads_right = [read.strip() for read in reads_right]

# remove first 3 lines from both left and right reads
reads_left = reads_left[3:]
reads_right = reads_right[3:]

# separate lines at tab character
reads_left = [read.split('\t') for read in reads_left]
reads_right = [read.split('\t') for read in reads_right]

# check if column 1 of left and right reads match
for i in range(len(reads_left)):
    if reads_left[i][0] != reads_right[i][0]:
        raise Exception('Error: reads are not in the same order')
    
# only use columns 2,3,4,5,6,10 and 11
reads_left = [[read[1], read[2], read[3], read[4], read[5], read[9], read[10]] for read in reads_left]
reads_right = [[read[1], read[2], read[3], read[4], read[5], read[9], read[10]] for read in reads_right]

# print(reads_left[0])
# print(reads_right[0])

# convert to numpy array of strings
reads_left = np.array(reads_left, dtype='str')
reads_right = np.array(reads_right, dtype='str')

# only keep reads with flag value 0 or 16 in left and flag value 0 or 16 in right
indices_left = np.where((reads_left[:,0] == '0') | (reads_left[:,0] == '16'))
indices_right = np.where((reads_right[:,0] == '0') | (reads_right[:,0] == '16'))
indices = np.intersect1d(indices_left, indices_right)
reads_left = reads_left[indices]
reads_right = reads_right[indices]

# print('Number of reads after flag filtering: ', len(reads_left))

# only keep reads with MAPQ value >= 70 in left and right
indices_left = np.where(reads_left[:,3].astype('int') >= 70)
indices_right = np.where(reads_right[:,3].astype('int') >= 70)
indices = np.intersect1d(indices_left, indices_right)
reads_left = reads_left[indices]
reads_right = reads_right[indices]

# print('Number of reads after MAPQ filtering: ', len(reads_left))

# only keep reads with CIGAR strings that do not contain 'I' or 'D' in left and right
indices = np.where((np.char.find(reads_left[:,4], 'I') == -1) & (np.char.find(reads_left[:,4], 'D') == -1) & (np.char.find(reads_right[:,4], 'I') == -1) & (np.char.find(reads_right[:,4], 'D') == -1))
reads_left = reads_left[indices]
reads_right = reads_right[indices]

In [134]:
# parse cigar string into binary string where 1 indicates a match and 0 indicates a mismatch
def parse_cigar(cigar_string):
    # get only numbers from cigar string
    numbers = re.findall(r'\d+', cigar_string)
    # get only letters from cigar string
    letters = re.findall(r'[A-Z]', cigar_string)
    # iterate through numbers
    sequence = ''
    for i in range(len(numbers)):
        # if letter is not a valid symbol, set number to 0
        if letters[i] == 'M':
            sequence += '1' * int(numbers[i])
        else:
            sequence += '0' * int(numbers[i])
    return sequence

# parse quality string into binary string where 1 indicates a high quality base and 0 indicates a low quality base
def parse_quality(quality_string):
    valid_symbols = ('?', "@", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I')
    sequence = ''
    for i in range(len(quality_string)):
        if quality_string[i] in valid_symbols:
            sequence += '1'
        else:
            sequence += '0'
    return sequence

# combine cigar and quality strings into one binary string
def parse_cigar_quality(cigar_string, quality_string):
    quality_seq = parse_quality(quality_string)
    cigar_seq = parse_cigar(cigar_string)
    sequence = ''
    for i in range(len(cigar_seq)):
        if cigar_seq[i] == '1' and quality_seq[i] == '1':
            sequence += '1'
        else:
            sequence += '0'
    return sequence

def parse_sequence(sequence, cigar_string, quality_string):
    valid_symbols = ('A', 'C', 'G', 'T')
    checked_seq = parse_cigar_quality(cigar_string, quality_string)
    parsed_sequence = ''
    for i in range(len(checked_seq)):
        if checked_seq[i] == '1' and sequence[i] in valid_symbols:
            parsed_sequence += sequence[i]
        else:
            parsed_sequence += '0'
    return parsed_sequence

def align_fragment(length_sequence, pos, cigar_string, sequence, quality_string):
    parsed_sequence = parse_sequence(sequence, cigar_string, quality_string)
    # get only numbers from cigar string
    numbers = re.findall(r'\d+', cigar_string)
    # get only letters from cigar string
    letters = re.findall(r'[A-Z]', cigar_string)
    offset = 0
    # iterate through letters till find M
    for i in range(len(letters)):
        if letters[i] != 'M':
            offset += int(numbers[i])
        else:
            break
    aligned_sequence = ''
    # add pos - 1 0s to the beginning of the sequence
    aligned_sequence += '0' * (pos - 1 - offset)
    # add parsed sequence to the end of the sequence
    aligned_sequence += parsed_sequence
    # add 0s to the end of the sequence
    aligned_sequence += '0' * (length_sequence - len(aligned_sequence))
    return aligned_sequence

# merge aligned sequences
def merge_sequences(left_sequence, right_sequence, length_sequence):
    merged_sequence = ''
    for i in range(length_sequence):
        if left_sequence[i] == right_sequence[i]:
            merged_sequence += left_sequence[i]
        # if left sequence is 0, add right sequence
        elif left_sequence[i] == '0':
            merged_sequence += right_sequence[i]
        # if right sequence is 0, add left sequence
        elif right_sequence[i] == '0':
            merged_sequence += left_sequence[i]
        # if left and right sequences are different, add 0
        else:
            merged_sequence += '0'

    return merged_sequence

In [135]:
print(reads_left.shape)
print(reads_right.shape)
print(reads_left[0])
print(reads_right[0])

(10, 7)
(10, 7)
['0' '5NL43' '21' '70' '6S70M'
 'GACCATATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCGATAAAGCCTGCCTTGAG'
 'A/AAAEEAEEEAEEEEEEEEEEEEAEAEEEAEEEAEEEE//EEE/EAEE/EE/EEEEEEEAEEEEAEEEEAE/EEE']
['16' '5NL43' '230' '70' '75M1S'
 'TCTCGACACAGGACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAT'
 '<E/EEEEAEEEE/E<EEEE/EEE/E/EAEAEEEEEEEEEAEAEEEEEEEEEEEEEEEEEEE/EEEEEEEEEAAAAA']


In [137]:
# load wildtype sequence
file_path_wildtype = '../data/experimental_data/5NL43.txt'
# set sequence length to length of wildtype sequence
with open(file_path_wildtype, 'r') as wildtype_file:
    # get line that doesn't start with '>'
    for line in wildtype_file:
        if not line.startswith('>'):
            sequence_length = len(line.strip())
            wildtype_sequence = line.strip()
            break
            

print(sequence_length)
print(wildtype_sequence)

535
GGTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTCAAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCAGTGGCGCCCGAACAGGGACTTGAAAGCGAAAGTAAAGCCAGAGGAGATCTCTCGACGCAGGACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAATTTTGACTAGCGGAGGCTAGAAGGAGAGAGATGGGTGCGAGAGCGTCGGTATTAAGCGGGGGAGAATTAGATAAATGGGAAAAAATTCGGTTAAGGCCAGGGGGAAAGAAACAATATAAACTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGATTCGCAGTTAATCCTGGCCTTTTAGAGACATCAGAAGGCTGTAGACAAATACTGGGACAGCTACAACCATC


In [138]:
left_seq = align_fragment(sequence_length, 21, '6S70M', 'GACCATATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCGATAAAGCCTGCCTTGAG', 'A/AAAEEAEEEAEEEEEEEEEEEEAEAEEEAEEEAEEEE//EEE/EAEE/EE/EEEEEEEAEEEEAEEEEAE/EEE')
right_seq = align_fragment(sequence_length, 230, '75M1S', 'TCTCGACACAGGACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAT', '<E/EEEEAEEEE/E<EEEE/EEE/E/EAEAEEEEEEEEEAEAEEEEEEEEEEEEEEEEEEE/EEEEEEEEEAAAAA')
print(left_seq)
print(right_seq)
print(wildtype_sequence)

00000000000000000000ATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGG00CCC0CTGC0TA0GCCTCGATAAAGCCTGCCT0GAG0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000C0CGACACAGG0C0CGGC0TGC0G0AGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGG0GAGTACGCCAAAA0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [139]:
print(merge_sequences(left_seq, right_seq, sequence_length))
print(wildtype_sequence)

00000000000000000000ATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGG00CCC0CTGC0TA0GCCTCGATAAAGCCTGCCT0GAG00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000C0CGACACAGG0C0CGGC0TGC0G0AGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGG0GAGTACGCCAAAA000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
GGTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTCAAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCAGTGGCGCCCGAACAGGGACTTGAAAGCGAAAGTAAAGCCAGAGGAGATCTCTCGACGCAGGACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAATTTTGACTAGCGGAGGCTAGAAGGAGAGAGATGGGTGCGAGAGCGTCGGTATTAAGCGGGGGAGAATTAGATAAATGGGAAAAAATTCGGTTAAGGCCAGGGGGAAAGAAACAATATAAACTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGA

In [140]:
print(reads_left[2])
print(reads_right[2])

['16' '5NL43' '414' '70' '71M5S'
 'AAACAATATAAACTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGATTCGCAGTTAATCCTGGCCTGTTAG'
 'EEEAEE6EEE/AEEEE/EEEEEAEEEEEEAEEE/EEEE/EEEEEEEEEEEEAEE/EEE/EE/EEEEE/EEEAAAAA']
['0' '5NL43' '324' '70' '76M'
 'AGAAGGAGAGAGATGGGAGCGAGAGCGTCGGGATTAAGCGGGGGAGAATTAGATAAATGGGAAAAAATTCGGTTAA'
 'AAA6AEAEA<E//EEAE/EAEEEEEEEEEAE/EEEEEAEEEEEEEEEEEEEEE</A/EEEEE/EEEEAEEEEEEEA']


In [141]:
left_seq = align_fragment(sequence_length, 414, reads_left[2][4], reads_left[2][5], reads_left[2][6])
right_seq = align_fragment(sequence_length, 324, reads_right[2][4], reads_right[2][5], reads_right[2][6])
print(left_seq)
print(right_seq)
print(wildtype_sequence)

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000AAACAA0ATA0ACTAA0ACATATAGTATGGGCA0GCAG0GAGCTAGAACGATTC0CAG0TA0TCCTG0CCT000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000AGA0GGAGA0A00TGGG0GCGAGAGCGTCGG0ATTAAGCGGGGGAGAATTAGA00A0TGGGA0AAAATTCGGTTAA00000000000000000000000000000000000000000000000000000000000000000

In [142]:
print(reads_left[3])
print(reads_right[3])

['16' '5NL43' '389' '70' '76M'
 'AATTCGGTTAAGGCCAGGGGGAAAGAAACAATATACACTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGA'
 'EEE/<6A/EEEEE<<<EEE//EE</EEAAEEEEAE/EA6EEEEEEA/E/<E//AEE/EAE//EEEEE6EEE/AAAA']
['0' '5NL43' '328' '70' '76M'
 'GGAGAGAGATGGGTGCGAGAGCGTCGGTATTAAGCGGGGGAGAATTAGATAAATGGGAAAAAATTCGGTTAAGGCC'
 'AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAAEEEEEE']


In [143]:
left_seq = align_fragment(sequence_length, 389, reads_left[3][4], reads_left[3][5], reads_left[3][6])
right_seq = align_fragment(sequence_length, 328, reads_right[3][4], reads_right[3][5], reads_right[3][6])
print(left_seq)
print(right_seq)
print(wildtype_sequence)

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000AAT000G0TAAGG000GGG00AA00AAACAATATA0AC0AAAACAT0T00T00GGG0AAG00GGGAG0TAG0ACGA00000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000GGAGAGAGATGGGTGCGAGAGCGTCGGTATTAAGCGGGGGAGAATTAGATAAATGGGAAAAAATTCGGTTAAGGCC0000000000000000000000000000000000000000000000000000000000000

In [144]:
print(reads_left[4])
print(reads_right[4])

['16' '5NL43' '445' '70' '4S61M11S'
 'TGGTCAAGCAGGGAGCTAGAACGATTCGCAGTTAATCCTGGCCTTTTAGAGACATCAGAAGGCTGCCGGCAAATAC'
 '<AE/EE<EE/EEAEE/EEAEEEEEEEEEEEEEEE/EEEEEAAEEEEEEEEEEEE6EEEEE/EEE<E/EAAEA/AAA']
['0' '5NL43' '397' '70' '2S74M'
 'GCTAAGGCCAGGGGGAAAGAAACAATATAAACTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGATTCGCA'
 'AAAAAAE/EAEEEEEEEEEAEEEEEEEAAEEEEEEEEEE6EEEAEEA/EAA/EEEEEEEEEEEAE/AEE//EEAEE']


In [145]:
left_seq = align_fragment(sequence_length, 445, reads_left[4][4], reads_left[4][5], reads_left[4][6])
right_seq = align_fragment(sequence_length, 397, reads_right[4][4], reads_right[4][5], reads_right[4][6])
print(left_seq)
print(right_seq)
print(wildtype_sequence)

000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000CA0GC0GGGAG0TAGAACGATTCGCAGTTA0TCCTGGCCTTTTAGAGACA0CAGAA0GCT0000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000TAAGG0CAGGGGGAAAGAAACAATATAAACTAAAACA0ATAGTAT0GGC0AGCAGGGAGCTAG0ACG0

In [146]:
# position describes FIRST MATCHING BASE not first base